# LLM agent Hackathon 

## Alireza Ghafarollahi, MIT

# Create your first agent
- ## LLM: gpt-4o
- ## Task: write a Python code
- ## Generation-reflection mechanism

In [1]:
import json
import os
import re

with open('config.json') as f:
    config = json.load(f)

OPENAI_API_KEY = config["api_key"]
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [2]:
from openai import OpenAI
import json
import os.path as osp
import subprocess

client = OpenAI(organization ='')

In [3]:
def get_response_from_llm(
        system_message,
        prompt,
        model,
        reasoning_effort="medium",
        print_debug=False,
        msg_history=None,
        temperature=0.75,
        client=client):

    if msg_history is None:
        msg_history = []

    new_msg_history = msg_history + [{"role": "user", "content": prompt}]
    
    if model in ["gpt-4o", "gpt-4-turbo"]:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "developer", "content": system_message},
                *new_msg_history,
            ],
            temperature=temperature,
            max_completion_tokens=15000
        )
        print(token_usage(response))

    elif model in ["gpt-4.1"]:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "developer", "content": system_message},
                *new_msg_history,
            ],
            temperature=temperature,
            max_completion_tokens=20000
        )
        print(token_usage(response))
        
    elif model in ["o1", "o1-mini", "o3", "o3-mini"]:
        response = client.chat.completions.create(
            model=model,
            reasoning_effort=reasoning_effort,
            messages=[
                {"role": "developer", "content": system_message},
                *new_msg_history,
            ],
            temperature=temperature,
            max_completion_tokens=20000
        )
        print(token_usage(response))
        
    content = response.choices[0].message.content
    new_msg_history = new_msg_history + [{"role": "assistant", "content": content}]

    if print_debug:
        print()
        print("*" * 20 + " LLM START " + "*" * 20)
        for j, msg in enumerate(new_msg_history):
            print(f'{j}, {msg["role"]}: {msg["content"]}')
            print("*" * 51)
        #print(content)
        print("*" * 21 + " LLM END " + "*" * 21)
        print()

    return content, new_msg_history

def token_usage(response):
    dic = {"prompt tokens": response.usage.prompt_tokens,
          "completion tokens" : response.usage.completion_tokens,
          "total tokens": response.usage.total_tokens,
           "reasoning tokens": response.usage.completion_tokens_details.reasoning_tokens
          }
    return dic

In [4]:
system_message = '''You are a senior coder with expertise in writing Python codes.'''

prompt_1 = ''' You carefully write a Python code for the given task.

INSTRUCTION:
- Always save the plots, rather than showing them.

Respond in the following format

<code_START>
<CODE>
<code_FINISH>

Replace <CODE> with your complete and well-organized Python script. 

Additional note:
Write just the raw Python logic, no Markdown formatting, and no main() function wrapper, no ```python in the beginning and ``` in the end.

TASK:
{task}
'''

prompt_2 = '''
Carefully check the code that was just created for completeness and accuracy. Ensure the code is free of error and addresses the given query.

Give this a lot of thought and then, revise the code if:
1- there are issues and errors.
2- you find room for improvement. 

Return the revised code in the following format

<code_START>
<CODE>
<code_FINISH>

Replace <CODE> with your revised Python script. If there is nothing to improve and the code looks fine, replace <CODE> with "NO_REFINEMENT".
'''

task = '''Derive the mathematical expression for the roots of a third degree polynomial. 
Then give a sample function and compute its roots using the expressions. Finally, plot the function, showing the roots and annotaing the mathematical expression.''' 

code_history = []

code_1, code_history = get_response_from_llm(system_message=system_message,
                                          model='gpt-4o',
                                          prompt=prompt_1.format(task=task),
                                          temperature=0,
                                          print_debug=False,
                                             msg_history=code_history)

code_2, code_history = get_response_from_llm(system_message=system_message,
                                          model='gpt-4o',
                                          prompt=prompt_2.format(),
                                          temperature=0,
                                          print_debug=True,
                                            msg_history=code_history)

{'prompt tokens': 161, 'completion tokens': 466, 'total tokens': 627, 'reasoning tokens': 0}

******************** LLM START ********************
0, user:  You carefully write a Python code for the given task.

INSTRUCTION:
- Always save the plots, rather than showing them.

Respond in the following format

<code_START>
<CODE>
<code_FINISH>

Replace <CODE> with your complete and well-organized Python script. 

Additional note:
Write just the raw Python logic, no Markdown formatting, and no main() function wrapper, no ```python in the beginning and ``` in the end.

TASK:
Derive the mathematical expression for the roots of a third degree polynomial. 
Then give a sample function and compute its roots using the expressions. Finally, plot the function, showing the roots and annotaing the mathematical expression.

***************************************************
1, assistant: <code_START>
import numpy as np
import matplotlib.pyplot as plt
from sympy import symbols, solve, latex

# Define 

In [5]:
match_2 = re.search(r'<code_START>(.*?)<code_FINISH>', code_2, re.DOTALL)
if match_2:
    code_block_2 = match_2.group(1).strip()
else:
    print("Code block not found.")

if re.search("NO_REFINEMENT", code_block_2):
    print('No refinement needed, executing code from coder_1')

    match_1 = re.search(r'<code_START>(.*?)<code_FINISH>', code_1, re.DOTALL)
    if match_1:
        code_block_1 = match_1.group(1).strip()
    else:
        print("Code block not found.")

    with open(f'python_code_agent_3.py', 'w') as f:
        f.writelines(code_block_1)
        
    command = ['python', f"python_code_agent_3.py"]            
        
    result = subprocess.run(command, stderr=subprocess.PIPE, text=True)

else:
    print('Code was refined, executing code from coder_2')
    
    with open(f'python_code_agent_3.py', 'w') as f:
        f.writelines(code_block_2)
        
    command = ['python', f"python_code_agent_3.py"]  
    
    result = subprocess.run(command, stderr=subprocess.PIPE, text=True)
    
    

Code was refined, executing code from coder_2
